In [1]:
import pandas as pd
import nltk
import numpy as np
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import SnowballStemmer
import time
import pickle

#Visualization
import matplotlib.pyplot as plt

#Feature Engineering
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#Evaluation Metric
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score, precision_score,recall_score,classification_report
import seaborn as sns
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
class SVM:
    def __init__(self, lambda_param=1e-4, epoch=1000, batch_size=256, tol=1e-4, random_state=42):
        self.lambda_param = lambda_param
        self.epoch = epoch
        self.batch_size = batch_size
        self.tol = tol
        self.random_state = random_state
        self.is_trained = False

    def fit(self, X, y):
        if hasattr(X, "toarray"):
            X = csr_matrix(X)
        self.num_samples, self.num_features = X.shape
        y_unique = np.unique(y)
        if len(y_unique) != 2:
            raise ValueError("Phân loại nhị phân cần 2 nhãn")
        if set(y_unique) == {0, 1}:
            y = np.where(y == 0, -1, 1)
        self.w = np.zeros(self.num_features, dtype=np.float32)
        self.b = 0.0
        np.random.seed(self.random_state)
        t = 0
        previous_objective = float("inf")
        for ep in range(1, self.epoch + 1):
            indices = np.random.permutation(self.num_samples)
            for start in range(0, self.num_samples, self.batch_size):
                t += 1
                end = start + self.batch_size
                batch_idx = indices[start:end]
                X_batch = X[batch_idx]
                y_batch = y[batch_idx]

                eta = 1.0 / (self.lambda_param * t)
                margins = y_batch * (X_batch.dot(self.w) + self.b)
                mask = margins < 1
                self.w *= (1 - eta * self.lambda_param)
                if np.any(mask):
                    X_violate = X_batch[mask]
                    y_violate = y_batch[mask]
                    self.w += (eta / self.batch_size) * np.dot(y_violate, X_violate.toarray() if hasattr(X_violate, "toarray") else X_violate)
                    self.b += (eta / self.batch_size) * np.sum(y_violate)
                norm_w = np.linalg.norm(self.w)
                factor = min(1, (1.0 / np.sqrt(self.lambda_param)) / (norm_w))
                self.w *= factor
            decision = X.dot(self.w) + self.b
            hinge_losses = np.maximum(0, 1 - y * decision)
            objective = 0.5 * self.lambda_param * np.dot(self.w, self.w) + np.mean(hinge_losses)
            if ep % 10 == 0:
                print(f"Epoch {ep}, Giá trị hàm mục tiêu: {objective:.4f}")
            if abs(previous_objective - objective) < self.tol:
                print(f"Dừng sớm tại epoch {ep}, giá trị hàm mục tiêu thay đổi: {abs(previous_objective - objective):.6f}")
                break
            previous_objective = objective
        self.is_trained = True
        return self

    def predict(self, X):
        if not self.is_trained:
            raise Exception("Mô hình chưa được huấn luỵen")

        if hasattr(X, "toarray"):
            X = csr_matrix(X)

        decision = X.dot(self.w) + self.b
        return np.where(decision >= 0, 1, 0)

In [3]:
a = [i for i in range(1000, 10000, 500)]
for i in a:
    df = pd.read_csv(f"https://media.githubusercontent.com/media/PTIT-Assignment-Projects/ai-svm-email-spam/refs/heads/main/dataset//sampled_dataset{i}.csv")
    def remove_special_characters(word):
        return re.sub(r'[^a-zA-Z\s]', '', word)
    ENGLISH_STOP_WORDS = set(stopwords.words('english'))
    def remove_stop_words(words):
        return [word for word in words if word not in ENGLISH_STOP_WORDS]
    def remove_url(word):
        return re.sub(r"http\S+", "", word)
    df['text'] = df['text'].apply(remove_special_characters)
    df['text'] = df['text'].apply(remove_url)
    df['text'] = df['text'].apply(word_tokenize)
    df['text'] = df['text'].apply(remove_stop_words)
    df['text'] = df['text'].apply(' '.join)
    stemmer = SnowballStemmer('english')
    def stem_text(text):
        tokens = nltk.word_tokenize(text)
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)
    df['text'] = df['text'].apply(stem_text)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['text'])
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    svm = SVM()
    start_time = time.time()
    svm.fit(X_train, y_train)
    end_time = time.time()
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    result = {
    'class_name': svm.__class__.__name__ + "_pegasos",
    'n': len(df),
    'time': end_time - start_time,
    'accuracy_score': accuracy,
    'f1_score': f1
    }
    results_df = pd.DataFrame([result])
    print(results_df)
    results_df.to_csv(f'svm_pegasos_{i}.csv', index=False)

Epoch 10, Giá trị hàm mục tiêu: 2.9615
Epoch 20, Giá trị hàm mục tiêu: 9.8139
Epoch 30, Giá trị hàm mục tiêu: 6.7243
Epoch 40, Giá trị hàm mục tiêu: 4.2905
Epoch 50, Giá trị hàm mục tiêu: 2.2824
Epoch 60, Giá trị hàm mục tiêu: 0.3856
Epoch 70, Giá trị hàm mục tiêu: 0.2783
Epoch 80, Giá trị hàm mục tiêu: 0.2139
Epoch 90, Giá trị hàm mục tiêu: 0.1710
Epoch 100, Giá trị hàm mục tiêu: 0.1417
Epoch 110, Giá trị hàm mục tiêu: 0.3392
Epoch 120, Giá trị hàm mục tiêu: 0.1321
Epoch 130, Giá trị hàm mục tiêu: 0.1122
Epoch 140, Giá trị hàm mục tiêu: 0.1160
Epoch 150, Giá trị hàm mục tiêu: 0.0867
Epoch 160, Giá trị hàm mục tiêu: 0.3270
Epoch 170, Giá trị hàm mục tiêu: 0.0835
Epoch 180, Giá trị hàm mục tiêu: 0.0751
Epoch 190, Giá trị hàm mục tiêu: 0.0797
Epoch 200, Giá trị hàm mục tiêu: 0.0707
Epoch 210, Giá trị hàm mục tiêu: 0.0645
Epoch 220, Giá trị hàm mục tiêu: 0.0604
Epoch 230, Giá trị hàm mục tiêu: 0.0681
Dừng sớm tại epoch 238, giá trị hàm mục tiêu thay đổi: 0.000075
    class_name     n     